In [3]:
import pandas as pd

def remove_duplicates_csv(input_file, output_file, column_name=None):
    """
    Remove duplicates from a CSV file
    
    Parameters:
    input_file (str): Path to input CSV file
    output_file (str): Path to output CSV file
    column_name (str): Specific column to check for duplicates (optional)
    """
    try:
        # Read the CSV file
        df = pd.read_csv(input_file)
        
        print(f"Original number of rows: {len(df)}")
        
        # Remove duplicates
        if column_name:
            # Remove duplicates based on specific column
            duplicates = df.duplicated(subset=[column_name], keep=False)
            duplicate_count = duplicates.sum()
            df_clean = df.drop_duplicates(subset=[column_name], keep='first')
        else:
            # Remove all duplicate rows
            duplicates = df.duplicated(keep=False)
            duplicate_count = duplicates.sum()
            df_clean = df.drop_duplicates()
        
        print(f"Number of duplicates found: {duplicate_count}")
        print(f"Number of rows after cleaning: {len(df_clean)}")
        
        # Save the cleaned data
        df_clean.to_csv(output_file, index=False)
        print(f"Cleaned data saved to: {output_file}")
        
        # Show duplicate details if any were found
        if duplicate_count > 0:
            if column_name:
                duplicate_values = df[df.duplicated(subset=[column_name], keep=False)][column_name].unique()
            else:
                duplicate_values = df[df.duplicated(keep=False)].index.tolist()
            
            print(f"Duplicate items: {duplicate_values}")
        
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage
if __name__ == "__main__":
    input_csv = "s.csv"  # Change to your input file
    output_csv = "sclean.csv"  # Change to your desired output file
    
    # Remove duplicates from entire row
    remove_duplicates_csv(input_csv, output_csv)
    
    # Or remove duplicates based on specific column (e.g., 'email')
    # remove_duplicates_csv(input_csv, output_csv, column_name='email')

Processed file saved as: coordinateskonbinis.csv
Successfully extracted coordinates from 120 out of 120 URLs

First 10 rows with extracted coordinates:
                                                link   latitude   longitude
0  https://www.google.com/maps/place/7-Eleven/dat...  34.998392  135.731755
1  https://www.google.com/maps/place/FamilyMart/d...  34.997308  135.735106
2  https://www.google.com/maps/place/7-Eleven/dat...  34.992463  135.732591
3  https://www.google.com/maps/place/FamilyMart/d...  34.992262  135.731381
4  https://www.google.com/maps/place/7-Eleven+-+K...  34.995055  135.738844
5  https://www.google.com/maps/place/7-Eleven+-+S...  35.002628  135.731489
6  https://www.google.com/maps/place/FamilyMart+S...  35.002391  135.732243
7  https://www.google.com/maps/place/Daily+Yamaza...  34.996395  135.738645
8  https://www.google.com/maps/place/LAWSON+Nishi...  34.994058  135.725993
9  https://www.google.com/maps/place/Lawson+Shijo...  35.003363  135.732189

Sample coor

In [4]:
import pandas as pd
import re

def extract_coordinates_from_url(url):
    """
    Extract latitude and longitude from Google Maps URL
    """
    # Check if url is actually a string (not NaN/float)
    if pd.isna(url) or not isinstance(url, str):
        return None, None
    
    # Pattern to match coordinates in the URL
    # Looks for patterns like !3d34.998392!4d135.731755
    pattern = r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)'
    
    match = re.search(pattern, url)
    if match:
        try:
            latitude = float(match.group(1))
            longitude = float(match.group(2))
            return latitude, longitude
        except ValueError:
            return None, None
    else:
        return None, None

def determine_image_src(url):
    """
    Determine imageSrc value based on substrings in the URL
    """
    if pd.isna(url) or not isinstance(url, str):
        return None
    
    url_lower = url.lower()
    
    if "eleven" in url_lower:
        return "eleven"
    elif "lawson" in url_lower:
        return "lawson"
    elif "yamazaki" in url_lower:
        return "yamazaki"
    elif "family" in url_lower:
        return "family"
    elif "ministop" in url_lower:
        return "ministop"
    else:
        return "konbini"

def process_csv(input_file, output_file=None):
    """
    Process CSV file to extract coordinates from Google Maps URLs and determine imageSrc
    """
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Extract coordinates from each URL in the 'link' column
    coordinates = df['link'].apply(extract_coordinates_from_url)
    
    # Split the coordinates into separate columns
    df['lat'] = coordinates.apply(lambda x: x[0] if x and x[0] is not None else None)
    df['lng'] = coordinates.apply(lambda x: x[1] if x and x[1] is not None else None)
    
    # Determine imageSrc based on substrings in the link
    df['imageSrc'] = df['link'].apply(determine_image_src)
    
    # Save to new file if output_file is specified
    if output_file:
        df.to_csv(output_file, index=False)
        print(f"Processed file saved as: {output_file}")
    else:
        df.to_csv(input_file, index=False)
        print(f"Original file updated: {input_file}")
    
    # Print summary
    successful_extractions = df['latitude'].notna().sum()
    total_rows = len(df)
    image_src_counts = df['imageSrc'].value_counts()
    
    print(f"Successfully extracted coordinates from {successful_extractions} out of {total_rows} URLs")
    print(f"\nImageSrc distribution:")
    for store_type, count in image_src_counts.items():
        print(f"  {store_type}: {count}")
    
    return df

# Process the file
input_filename = "sclean.csv"
output_filename = "scoords.csv"

result_df = process_csv(input_filename, output_filename)

# Display results
print("\nFirst 10 rows with extracted coordinates and imageSrc:")
print(result_df[['link', 'latitude', 'longitude', 'imageSrc']].head(10))

print(f"\nSample coordinates and imageSrc from first 5 valid URLs:")
valid_coords = result_df[result_df['latitude'].notna()][['latitude', 'longitude', 'imageSrc']].head()
for idx, row in valid_coords.iterrows():
    print(f"Row {idx}: Lat {row['latitude']}, Lng {row['longitude']}, ImageSrc: {row['imageSrc']}")

Processed file saved as: scoords.csv


KeyError: 'latitude'